# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [29]:
from pathlib import Path

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY1"]=os.getenv("GROQ_API_KEY1")


config_list = [
    {
        # Let's choose the Mixtral 8x22B model
        #"model": "llama3-8b-8192",
        # "model": "llama-3.3-70b-versatile",
        "model": "deepseek-r1-distill-llama-70b",
        # Provide your Mistral AI API key here or put it into the MISTRAL_API_KEY environment variable.
        "api_key": os.environ.get("GROQ_API_KEY1"),
        # We specify the API Type as 'mistral' so it uses the Mistral AI client class
        "api_type": "groq",
    }
]

llm_config={"config_list": config_list}
print(llm_config)


{'config_list': [{'model': 'deepseek-r1-distill-llama-70b', 'api_key': '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', 'api_type': 'groq'}]}


In [30]:
from autogen import ConversableAgent

## Creating the needed agents

In [31]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [32]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [33]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [34]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [35]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [36]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
customer_proxy_agent (to Onboarding Personal Information Agent):



--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):


Alright, let's break down what's happening here. The user provided a history where I asked for their name and location. They responded, but I can't see their exact reply because the query is incomplete. 

Hmm, in the current query, the user just wrote "Okay, so". It seems like they'r

/Users/sunjsg/anaconda3/envs/env310finrobot/lib/python3.10/site-packages/autogen/oai/groq.py:282: UserWarning: Cost calculation not available for model deepseek-r1-distill-llama-70b
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


customer_proxy_agent (to Onboarding Personal Information Agent):

SUNDARA and Bangalore

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 

Alright, let's figure out the user's needs here. They provided the name "SUNDARA" and the location "Bangalore". They want the information returned as a JSON object with 'name' and 'location' keys.

I should make sure to format the JSON correctly. Also, I notice the user mentioned to return it as a JSON object only, so I need to avoid any extra text or explanations.

I'll structure the JSON with the provided name and location. Since "SUNDARA" is in uppercase, I'll include it as-is unless ins

## Print out the summary

In [37]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'content': '\nAlright, let\'s figure out the user\'s needs here. They provided the name "SUNDARA" and the location "Bangalore". They want the information returned as a JSON object with \'name\' and \'location\' keys.\n\nI should make sure to format the JSON correctly. Also, I notice the user mentioned to return it as a JSON object only, so I need to avoid any extra text or explanations.\n\nI\'ll structure the JSON with the provided name and location. Since "SUNDARA" is in uppercase, I\'ll include it as-is unless instructed otherwise. "Bangalore" is the location, so I\'ll add that accordingly.\n\nDouble-checking to ensure the JSON syntax is correct and there are no errors. Once that\'s confirmed, I can present the JSON response as requested. Finally, as per the instructions, I\'ll include \'TERMINATE\' to indicate the end of the interaction.\n</think>\n\n**Step 1:** Return the customer information as a JSON object in the format `{\'name\': \'\', \'location\': \'\'}`.\n\n**Step 2:** Aft

## Print out the cost

In [ ]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")